In [ ]:
pip install kfp scikit-learn==1.0.1 mlflow==1.21.0 dill==0.3.4

### Component 만들기 위한 규칙

1. 컴포넌트 콘텐츠에서 필요한 값들은 Config로 정의
   > Kubeflow에서는 파이프라인을 Kubeflow 포맷으로 변환할 때, 컴포넌트 간의 연결에서 정해진 입력과 출력의 타입이 일치하는지 체크합니다. 만약 컴포넌트가 필요로 하는 입력과 다른 컴포넌트로부터 전달받은 출력의 포맷이 일치하지 않을 경우 파이프라인 생성을 할 수 없습니다.
2. argument의 타입 힌트 작성
   > Kubeflow에서 반환 값으로 사용할 수 있는 타입은 json에서 표현할 수 있는 타입들만 사용할 수 있습니다. 대표적으로 사용되며 권장하는 타입들은 다음과 같습니다.
3. 만약 단일 값이 아닌 여러 값을 반환하려면 collections.namedtuple 이용


### Docker로 Component 작성
아래의 파이썬 코드와 Docker 코드는 동일한 기능을 수행함.
- docker pull python:3.7
- print(number)



In [3]:
from kfp.components import create_component_from_func

@create_component_from_func
def print_and_return_number(number: int) -> int:
    print(number)
    return number

if __name__ == "__main__":
    print_and_return_number.component_spec.save("print_and_return_number.yaml")

In [4]:
from kfp.components import load_component_from_file

print_and_return_number = load_component_from_file("print_and_return_number.yaml")

### Pipeline 구동에 필요한 yaml 생성하기

In [6]:
@create_component_from_func
def sum_and_print_numbers(number_1: int, number_2: int) -> int:
    sum_num = number_1 + number_2
    print(sum_num)
    return sum_num

### 파이프라인 정의하기
* 파이프라인은 여러개의 component로 구성 됨.
* `set_display_name`를 사용하면 run page의 네모박스에 표현할 단어를 넣을 수 있음.

In [7]:
from kfp.dsl import pipeline


@pipeline(name="example_pipeline")
def example_pipeline(number_1: int, number_2: int):
    number_1_result = print_and_return_number(number_1).set_display_name("This is number 1")
    number_2_result = print_and_return_number(number_2).set_display_name("This is number 2")
    sum_result = sum_and_print_numbers(
        number_1=number_1_result.output, number_2=number_2_result.output
    ).set_display_name("This is sum of number 1 and number 2")

In [8]:
if __name__ == "__main__":
    import kfp
    kfp.compiler.Compiler().compile(example_pipeline, "example_pipeline.yaml")

### 생성한 yaml은 pipeline 페이지에서 불러오면 된다.

### 끝